In [ ]:
import numpy as np
import random
import pandas as pd

In [ ]:
def empty_columns(A):
    """Test whether a given boardstate has legal move
    ======
    Find all columns with 0 at the top

    Parameters
    ----------
    A : numpy array, int
        6*7 matrix, represent a given board state: 1 means Player 1, 2 means Player2, 0 means empty.

    Returns
    -------
    empty_indices : list, int
        Index of columns that are not full.
    """
    empty = (A[0,:]==0)
    return [i for i in range(len(empty)) if empty[i]]

In [ ]:
def check_winning(A,connectcount):
    """Test whether a given boardstate has a winner
    ======
    Find #1 or #2 in the given matrix reaches the given connectcount value.

    Parameters
    ----------
    A : numpy array, int
        6*7 matrix, represent a given board state: 1 means Player 1, 2 means Player2, 0 means empty.
    connectcount : int
        # of 1 or 2 in a line required for win.

    Returns
    -------
    w: int
    1 : Player 1 win.
    2 : Player 2 win.
    0 : There is no winner in the current boardstate.
    """
    for i in range(np.size(A,0)):
        for j in range(np.size(A,1)):
            if A[i,j]==0: #No need to waste time if the spot is empty
                continue

            if i<(np.size(A,0)-connectcount+1): #That is, if we are far enough from the bottom for a connect-X
                #Check columns below i,j
                if np.allclose(A[i:i+connectcount,j],np.ones(connectcount)):
                    return 1
                elif np.allclose(A[i:i+connectcount,j],(np.ones(connectcount)*2)):
                    return 2

                #Check down-right diagonals, if able
                if j<(np.size(A,1)-connectcount+1):
                    if np.allclose(np.diag(A[i:i+connectcount,j:j+connectcount]),np.ones(connectcount)):
                        return 1
                    elif np.allclose(np.diag(A[i:i+connectcount,j:j+connectcount]),(np.ones(connectcount)*2)):
                        return 2
                elif j>(connectcount-2): #Check down-left diagonals, if able
                    if np.allclose(np.diag(np.fliplr(A[i:i+connectcount,j-connectcount+1:j+1])),np.ones(connectcount)):
                        return 1
                    elif np.allclose(np.diag(np.fliplr(A[i:i+connectcount,j-connectcount+1:j+1])),(np.ones(connectcount)*2)):
                        return 2

            if j<(np.size(A,1)-connectcount+1): #Check rows to the right if able
                if np.allclose(A[i,j:j+connectcount],np.ones(connectcount)):
                    return 1
                elif np.allclose(A[i,j:j+connectcount],np.ones(connectcount)):
                    return 2
    return 0


In [ ]:
def generate_policy(A,turn,connectcount=4):
    """Randomly pick an available column to make the next move and determine whose turn next.
    ======
    Use the # of turn to determine the game state. If the game doesn't end, then generate a random move and switch turns.

    Parameters
    ----------
    A : numpy array, int
        6*7 matrix, represent a given board state: 1 means Player 1, 2 means Player2, 0 means empty.
    connectcount : int
        # of 1 or 2 in a line required for win.
    turn: int
        Represent the game state: 1, player 1 take the next move; 2, player 2 take the next move; 3, player 1 win;
        4, player 2 win; 5, a tie, i.e. no avaiable move but no one wins.
    policy: int
        A randomly generated integer represent the index of the next move column.

    Returns
    -------
    [A,turn]: list,
    A: 6*7 matrix, boardstate updated; turn: int, next turn updated.
    """
    if turn>2:
      print("Game is over!")
      if turn==3:
            print("Player 1 wins")
      elif turn==4:
            print("Player 2 wins")
      elif turn==5:
            print("It's a tie!")
      return [A,turn]
    w = check_winning(A,connectcount)
    if w==1:
        print("Game is over!")
        print("Player 1 wins")
        return [A,3]
    if w==2:
        print("Game is over!")
        print("Player 2 wins")
        return [A,4]
    empty_indices = empty_columns(A)
    if len(empty_indices)==0:
        print("Game is over!")
        print("It's a tie!")
        return [A,5]
    policy = empty_indices[random.randint(0,len(empty_indices)-1)]
    [A,turn] = generate_boardstate(A, turn, policy)
    return [A,turn]

In [ ]:
def generate_boardstate(A,turn,policy):
    """Generate the next boardstate.
    ======
    Use policy to update the boardstate and switich turn.

    Parameters
    ----------
    A : numpy array, int
        6*7 matrix, represent a given board state: 1 means Player 1, 2 means Player2, 0 means empty.
    turn: int
        Represent the game state: 1, player 1 take the next move; 2, player 2 take the next move; 3, player 1 win;
        4, player 2 win; 5, a tie, i.e. no avaiable move but no one wins.
    policy: int
        A randomly generated integer represent the index of the next move column.

    Returns
    -------
    [A,turn]: list,
    A: 6*7 matrix, boardstate updated; turn: int, next turn updated.
    """
    if turn<3:
        m = np.size(A,0)
        for i in range(m):
            if A[m-i-1,policy]==0:
                A[m-i-1,policy]=turn
                if turn==1:
                    turn=2
                elif turn==2:
                    turn=1
                return [A,turn]
        return "Error - Bad Policy choice!"
    return "The game is already over! Turn count too high"

In [ ]:
def generate_random_boardstate(turncount,rows=6,columns=7):
    """Randomly simulate the first n move of a empty boardstate.
    ======

    Parameters
    ----------
    turncount: int
    # of steps required to simulate.

    Returns
    -------
    [A,turn]: list,
    A: 6*7 matrix, boardstate updated; turn: int, next turn updated.
    """

    restart = True

    while restart:

      restart = False
      A = np.zeros((rows,columns))
      turn = 1

      for i in range(0,turncount):
        [A,turn]=generate_policy(A,turn)
        if check_winning(A,4)!=0:
          restart = True
          break
    return [A,turn]

In [ ]:
[A,turn]=generate_random_boardstate(41)
print(A)
# print(np.reshape(A, (1,42))[0])
print(turn)

[[2. 1. 2. 1. 1. 0. 2.]
 [2. 2. 2. 2. 2. 1. 1.]
 [1. 1. 1. 2. 1. 2. 2.]
 [1. 2. 2. 1. 1. 1. 2.]
 [1. 1. 2. 1. 2. 2. 1.]
 [2. 2. 1. 1. 1. 2. 1.]]
2


In [ ]:
# dataset = pd.read_csv('connect-4.csv')
# dataset = np.array(dataset)
# first_player = (dataset[0]=='x').astype(int)
# second_player = 2*(dataset[0]=='o').astype(int)
# d = np.reshape((first_player+second_player)[0:42],(7,6)).T
# print(d)
# print(dataset[0])

In [ ]:
check_winning(np.array([
 [2, 1, 2, 1, 1, 0, 2],
 [2, 2, 2, 2, 2, 1, 1],
 [1, 1, 1, 2, 1, 2, 2],
 [1, 2, 2, 1, 1, 1, 2],
 [1, 1, 2, 1, 2, 2, 1],
 [2, 2, 1, 1, 1, 2, 1]]),4)

0